# Conversion to other organisms

Most of the prior knowledge stored inside `Omnipath` is derived from human data, therefore they use gene names. Despite this, using homology we can convert gene names to other organisms.

To showcase how to do it inside `decoupler`, we will load the `MSigDB` database and convert it into gene symbols for mouse and fly.

Omnipathに保存されている事前知識のほとんどはヒトのデータに由来しており、そのため遺伝子名を使用している。しかし、相同性を利用すれば、遺伝子名を他の生物に変換することができる。

ここでは、MSigDBデータベースを読み込み、マウスとハエの遺伝子記号に変換する。

In [1]:
import decoupler as dc

In [2]:
msigdb = dc.get_resource('MSigDB')

In [3]:
msigdb.head()

,genesymbol,collection,geneset
0,MAFF,chemical_and_genetic_perturbations,BOYAULT_LIVER_CANCER_SUBCLASS_G56_DN
1,MAFF,chemical_and_genetic_perturbations,ELVIDGE_HYPOXIA_UP
2,MAFF,chemical_and_genetic_perturbations,NUYTTEN_NIPP1_TARGETS_DN
3,MAFF,immunesigdb,GSE17721_POLYIC_VS_GARDIQUIMOD_4H_BMDC_DN
4,MAFF,chemical_and_genetic_perturbations,SCHAEFFER_PROSTATE_DEVELOPMENT_12HR_UP


For this example we will filter by the `hallmark` gene sets collection:

この例では、ホールマーク遺伝子セットコレクションでフィルターをかける：

In [4]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection'] == 'hallmark']

In [5]:
# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]

In [6]:
msigdb.head()

,genesymbol,collection,geneset
233,MAFF,hallmark,HALLMARK_IL2_STAT5_SIGNALING
250,MAFF,hallmark,HALLMARK_COAGULATION
270,MAFF,hallmark,HALLMARK_HYPOXIA
373,MAFF,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB
377,MAFF,hallmark,HALLMARK_COMPLEMENT


Then, we can easily transform the obtained resource into mouse genes. Organisms can be defined by their common name, latin name or [NCBI Taxonomy identifier](https://www.ncbi.nlm.nih.gov/taxonomy).
    
Translating to an organism for the first time might take a while (~ 15 minutes). Since the data is stored in cache, the next times is going to run faster. If you need to reset the cache, run `rm -r .pypath/cache/`.

そして、得られたリソースを簡単にマウス遺伝子に変換することができる。生物は一般名、ラテン語名、NCBI分類学識別子で定義できる。

初回の生物への変換には時間がかかるかもしれない（～15分）。データはキャッシュに保存されるため、次回以降はより高速に実行されます。キャッシュをリセットする必要がある場合は、rm -r .pypath/cache/.

In [7]:
# Translate targets
mouse_msigdb = dc.translate_net(
    msigdb,
    target_organism='mouse',
    unique_by=('geneset', 'genesymbol')
)

/Users/tomoya/mambaforge/envs/py_scvi/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/tomoya/mambaforge/envs/py_scvi/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [8]:
mouse_msigdb.head()

,genesymbol,collection,geneset
0,Maff,hallmark,HALLMARK_IL2_STAT5_SIGNALING
1,Maff,hallmark,HALLMARK_COAGULATION
2,Maff,hallmark,HALLMARK_HYPOXIA
3,Maff,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB
4,Maff,hallmark,HALLMARK_COMPLEMENT


Note that when performing homology convertion we might gain or lose some genes from one organism to another.

Let us try the fruit fly (`7227`) now:

相同性変換を行う場合、ある生物から別の生物へ遺伝子を得たり失ったりする可能性があることに注意。

では、ミバエ(7227)を試してみよう：

In [9]:
# Translate targets
fly_msigdb = dc.translate_net(
    msigdb, target_organism=7227,
    unique_by=('genesymbol', 'geneset')
)

In [10]:
fly_msigdb.head()

,genesymbol,collection,geneset
0,betaTub85D,hallmark,HALLMARK_UNFOLDED_PROTEIN_RESPONSE
1,betaTub85D,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB
2,Rel,hallmark,HALLMARK_INTERFERON_GAMMA_RESPONSE
3,Rel,hallmark,HALLMARK_UV_RESPONSE_DN
4,Rel,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB


The `translate_net` function provides finer control, but in most cases it's enough to pass the name of the desired organism to the functions that download the data:

translate_net関数を使えばより細かい制御が可能だが、ほとんどの場合、データをダウンロードする関数に目的の生物の名前を渡すだけで十分だ：

In [17]:
spw = dc.get_resource('SignaLink_pathway', organism='rat')

In [18]:
spw.head()

,genesymbol,pathway
0,<NA>,T-cell receptor
1,<NA>,Receptor tyrosine kinase
2,<NA>,T-cell receptor
3,Tab2,Toll-like receptor
4,Tab2,Innate immune pathways


In [15]:
spw.shape

(1115, 2)

In [20]:
spw.query('genesymbol.notna()').shape

(309, 2)

PROGENy and CollecTRI have their own dedicated functions which work a similar way:

PROGENyとCollecTRIには、同様の働きをする専用の機能がある：

In [13]:
dc.get_progeny(organism='Mus musculus')

,source,target,weight,p_value
0,Androgen,Tmprss2,11.490631,0.000000e+00
1,Androgen,Nkx3-1,10.622551,2.242078e-44
2,Androgen,Mboat2,10.472733,4.624285e-44
3,Androgen,Sarg,11.386852,2.790209e-40
4,Androgen,Slc38a4,7.363805,1.253072e-39
...,...,...,...,...
1395,p53,Gm527,5.419560,2.742921e-11
1396,p53,Hepacam,4.416313,4.361244e-11
1397,p53,Rasal1,1.895177,5.096675e-11
1398,p53,Atp6ap2,2.461729,5.791803e-11


In [14]:
dc.get_collectri(organism='mouse')

0.00B [00:00, ?B/s]

,source,target,weight,PMID
0,Myc,Tert,1,10022128;10491298;10606235;10637317;10723141;1...
1,Spi1,Bglap,1,10022617
2,Spi1,Bglap3,1,10022617
3,Spi1,Bglap2,1,10022617
4,Smad3,Jun,1,10022869;12374795
...,...,...,...,...
38660,Runx1,Lcp2,1,20019798
38661,Runx1,Prr5l,1,20019798
38662,Twist1,Gli1,1,11948912
38663,Usf1,Nup188,1,22951020
